In [ ]:
import tkinter as tk
from tkinter import ttk
import time
import random
import threading
import redis

# Configuración de Redis
r = redis.Redis(
    host='localhost',
    port=6379,
    db=0)

# Simulación de los jugadores
jugadores = [
    "TheDestroyer", "Lightning", "Shadow", "IronFist", "Phantom", "Speedster", "Vortex", "Blaze", "Crusher", "Dragon", 
    "Rogue", "Sniper", "Zephyr", "Ghost", "Titan", "Raider", "Warrior", "Hunter", "Joker", "Samurai", 
    "Raven", "Demon", "Ninja", "Wolverine", "Maverick", "Thunder", "Falcon", "Storm", "Eagle", "Predator", 
    "Cyborg", "Stryker", "Phoenix", "Reaper", "Hawk", "Viper", "Sphinx", "Assassin", "Knight", "Berserker", 
    "Juggernaut", "Sorcerer", "Paladin", "Monk", "Vigilante", "Wraith", "Gladiator", "Barbarian", "Valkyrie", "Sentinel"
]

# Función para obtener el rango de un jugador según sus puntos
def obtener_rango_jugador(puntos):
    if puntos <= 1000:
        return "Básico"
    elif puntos <= 3000:
        return "Intermedio"
    elif puntos <= 5000:
        return "Avanzado"
    else:
        return "Legendario"

# Actualiza los puntajes de los jugadores
def actualizar_puntos(jugador_nombre):
    puntos_actuales = r.zscore("ranking_jugadores", jugador_nombre)
    if puntos_actuales is None:
        puntos_actuales = 0
    nuevos_puntos = random.randint(100, 6000)
    puntos_finales = max(puntos_actuales + nuevos_puntos, 0)
    timestamp = time.time()
    
    # Actualizar puntajes en Redis
    r.zadd("ranking_jugadores", {jugador_nombre: puntos_finales})
    r.hset(f"{jugador_nombre}_last_update", "timestamp", timestamp)
    
    # Guardar en el historial
    historial_entrada = f"{time.ctime(timestamp)}: {puntos_finales}"
    r.lpush(f"{jugador_nombre}_historial", historial_entrada)

def mostrar_historial(jugador_nombre):
    historial = r.lrange(f"{jugador_nombre}_historial", 0, -1)  # Obtener todo el historial
    return [entrada.decode('utf-8') for entrada in historial]

# Función para mostrar el top 3 jugadores con sus rangos
def mostrar_top3_ranking():
    ranking = r.zrevrange("ranking_jugadores", 0, 2, withscores=True)  # Top 3 jugadores
    top3 = []
    for nombre, puntos in ranking:
        jugador_nombre = nombre.decode('utf-8')
        puntos_int = int(puntos)
        rango = obtener_rango_jugador(puntos_int)
        top3.append((jugador_nombre, puntos_int, rango))
    return top3

# Hilo para actualizar puntajes en segundo plano
def actualizar_puntajes_periodicamente():
    contador = 0
    while contador < 50:
        jugador_nombre = random.choice(jugadores)
        actualizar_puntos(jugador_nombre)
        contador += 1
        time.sleep(10)

# Hilo para actualizar la interfaz gráfica con rangos
def actualizar_interfaz():
    while True:
        top3 = mostrar_top3_ranking()
        for i, (nombre, puntos, rango) in enumerate(top3):
            etiquetas[i].config(text=f"{i+1}. {nombre}: {puntos} puntos - {rango}")
        time.sleep(1)

# Configuración de la interfaz con Tkinter y ttk
def crear_interfaz():
    global etiquetas

    ventana = tk.Tk()
    ventana.title("Ranking de Jugadores")
    ventana.geometry("400x350")
    ventana.configure(bg="#1a1a1a")  # Fondo oscuro
    
    # Usar ttk para widgets modernos
    style = ttk.Style()
    style.theme_use("clam")  # Estilo moderno
    style.configure("TLabel", foreground="#ecf0f1", background="#1a1a1a", font=("Helvetica", 14))
    
    # Título estilizado
    titulo = ttk.Label(ventana, text="Top 3 Jugadores", font=("Helvetica", 20, "bold"))
    titulo.pack(pady=20)
    
    # Etiquetas para mostrar el top 3
    etiquetas = [ttk.Label(ventana, text="Cargando...", padding=10) for _ in range(3)]
    for etiqueta in etiquetas:
        etiqueta.pack(pady=5)
    
    # Botón para cerrar la ventana
    boton_cerrar = ttk.Button(ventana, text="Cerrar", command=ventana.destroy)
    boton_cerrar.pack(pady=20)
    
    # Hilos
    hilo_actualizar_puntajes = threading.Thread(target=actualizar_puntajes_periodicamente, daemon=True)
    hilo_actualizar_puntajes.start()
    hilo_interfaz = threading.Thread(target=actualizar_interfaz, daemon=True)
    hilo_interfaz.start()
    
    # Iniciar la interfaz gráfica
    ventana.mainloop()

# Ejecutar la interfaz
crear_interfaz()

In [ ]:

# Función para obtener la posición de un jugador en la tabla
def get_player_rank(player_id):
    rank = r.zrevrank("ranking_jugadores", player_id)
    if rank is not None:
        rank += 1  # Ajustar a base 1
        print(f'{player_id} está en la posición {rank}')
    else:
        print(f'{player_id} no se encuentra en la clasificación')
    return rank

# Función para obtener la puntuación de un jugador
def get_player_score(player_id):
    score = r.zscore("ranking_jugadores", player_id)
    if score is not None:
        print(f'{player_id} tiene una puntuación de {score}')
    else:
        print(f'{player_id} no tiene una puntuación registrada')
    return score

# Función para obtener el jugador en una posición determinada
def get_player_in_position(n):
    player = r.zrevrange("ranking_jugadores", n - 1, n - 1)  # Ajustar a base 0
    if player:
        print(f'El jugador en la posición {n} es {player[0].decode("utf-8")}')
    else:
        print(f'No hay ningún jugador en la posición {n}')
    return player[0].decode('utf-8') if player else None

# Resumen de jugadores
def resumen_jugadores():
    # Obtener todos los jugadores y sus puntajes
    jugadores_puntajes = r.zrange("ranking_jugadores", 0, -1, withscores=True)
    
    # Calcular el número total de jugadores
    total_jugadores = len(jugadores_puntajes)
    
    # Calcular el puntaje promedio
    if total_jugadores > 0:
        puntaje_total = sum(puntaje for jugador, puntaje in jugadores_puntajes)
        puntaje_promedio = puntaje_total / total_jugadores
    else:
        puntaje_promedio = 0
    
    print(f"Total de jugadores: {total_jugadores}")
    print(f"Puntaje promedio: {puntaje_promedio:.2f} puntos")

In [ ]:
get_player_rank("Joker")
get_player_score("Joker")
get_player_in_position(5)
resumen_jugadores()